# Análise da corrida
Com base no arquivo de entrada que contém os dados da corrida vamos extrair os seguintes resultados:
* Posição Chegada, Código Piloto, Nome Piloto, Qtde Voltas Completadas e Tempo Total de Prova.
##### Bônus
* Descobrir a melhor volta de cada piloto
* Descobrir a melhor volta da corrida
* Calcular a velocidade média de cada piloto durante toda corrida
* Descobrir quanto tempo cada piloto chegou após o vencedor

Para isso usaremos o Python e a biblioteca de análise de dados Pandas.

In [1]:
import pandas as pd

### Formatando o arquivo de entrada

Antes de começarmos o log fornecido foi copiado e colado em um arquivo ('corrida') sem extensão para podermos manipula-lo.
O arquivo campos separados por espaços em branco e outros separados por tabulação (\t), dessa forma vamos tratar o arquivo, trocando todas as tabulações por espaços e salvando um novo arquivo ('corrida2').


In [59]:
texto = []
# Abre o arquivo e carrega todas as linhas substituindo \t por um espaço em branco
with open('corrida', 'r', encoding='utf-8') as data_corrida:
    for line in data_corrida:
        texto.append(line.replace('\t', ' '))

# Salva o novo arquivo com o texto formatado        
with open('corrida2', 'w', encoding='utf-8') as arquivo:
    arquivo.writelines(texto)

### Carregando os dados no Pandas

Vamos carregar o novo arquivo no pandas utilizando o recurso *read_csv*, indicar o nome que cada coluna receberá, e por fim excluir o primeiro registro que representa o header do arquivo.
O processo foi feito dessa forma pois ao definir no pandas que o separador entre as colunas é o espaço, os nomes dos campos no header serão devidos em cada um de seus espaços, fazendo necessária a limpeza no dataframe.

In [79]:
# Definindo o nome das colunas
names =[
    'hora', 'n_piloto', '-', 'piloto', 'n_volta', 'tempo_volta',
    'velocidade_media_volta', '1', '2', '3'
]

# Carregando as informações no pandas
df = pd.read_csv('corrida2', delim_whitespace=True, names=names)
# Deletando as colunas ['-', '1', '2', '3'] que não tem nem uma utilidade para nós
df = df.drop(columns=['-', '1', '2', '3'])
# Deletando a primeira linha do dataframe, nela estão as informações do header que descartamos.
df = df.drop([0], axis=0)

# Imprimindo o dataframe.
df

,hora,n_piloto,piloto,n_volta,tempo_volta,velocidade_media_volta
1,23:49:08.277,038,F.MASSA,1,1:02.852,"44,275"
2,23:49:10.858,033,R.BARRICHELLO,1,1:04.352,"43,243"
3,23:49:11.075,002,K.RAIKKONEN,1,1:04.108,"43,408"
4,23:49:12.667,023,M.WEBBER,1,1:04.414,"43,202"
5,23:49:30.976,015,F.ALONSO,1,1:18.456,"35,47"
6,23:50:11.447,038,F.MASSA,2,1:03.170,"44,053"
7,23:50:14.860,033,R.BARRICHELLO,2,1:04.002,"43,48"
8,23:50:15.057,002,K.RAIKKONEN,2,1:03.982,"43,493"
9,23:50:17.472,023,M.WEBBER,2,1:04.805,"42,941"
10,23:50:37.987,015,F.ALONSO,2,1:07.011,"41,528"


### Formatando o DataFrame

Nesse ponto o dataframe está carregado com as informações.
Alguns detalhes devem ser modificados para obter o resultado desejado:
* O nome do piloto F.MASSA está escrito errado em uma das linhas.
* Transformar o n_volta em um valor numérico, pois no momento todos os campos são objects
* Formatar o campo tempo_volta para time_delta, isso permite operações com tempo de forma mais fácil
* Formatar o campo velocidade_media_volta para numérico, assim podemos descobrir a velocidade média na corrida

Todos os campos foram carregados no pandas como Object, é preciso transformar os campos para realizar as operações aritméticas desejadas.


In [80]:
# Corrigindo o erro no nome "F.MASS"
df['piloto'] = df['piloto'].replace('F.MASS', 'F.MASSA')
# Transformando o n_volta em numérico
df['n_volta'] = pd.to_numeric(df['n_volta'])
# Adicionando o '00:' ao tempo para ficar no padrão timedelta
df['tempo_volta'] = '00:' + df['tempo_volta']
# Transformando em time_delta
df['tempo_volta'] = pd.to_timedelta(df['tempo_volta'])
# Substituindo a virgula por ponto para ficar no padrão numérico
df['velocidade_media_volta'] = df['velocidade_media_volta'].str.replace(',', '.')
# Transformando em numérico
df['velocidade_media_volta'] = pd.to_numeric(df['velocidade_media_volta'])

df

,hora,n_piloto,piloto,n_volta,tempo_volta,velocidade_media_volta
1,23:49:08.277,038,F.MASSA,1,00:01:02.852000,44.275
2,23:49:10.858,033,R.BARRICHELLO,1,00:01:04.352000,43.243
3,23:49:11.075,002,K.RAIKKONEN,1,00:01:04.108000,43.408
4,23:49:12.667,023,M.WEBBER,1,00:01:04.414000,43.202
5,23:49:30.976,015,F.ALONSO,1,00:01:18.456000,35.470
6,23:50:11.447,038,F.MASSA,2,00:01:03.170000,44.053
7,23:50:14.860,033,R.BARRICHELLO,2,00:01:04.002000,43.480
8,23:50:15.057,002,K.RAIKKONEN,2,00:01:03.982000,43.493
9,23:50:17.472,023,M.WEBBER,2,00:01:04.805000,42.941
10,23:50:37.987,015,F.ALONSO,2,00:01:07.011000,41.528


## Obtendo os resultados espeados
* Posição Chegada, Código Piloto, Nome Piloto, Qtde Voltas Completadas e Tempo Total de Prova.

In [62]:
# Agrupando o dataframe por n_piloto e piloto
agrupado = df.groupby(['n_piloto', 'piloto' ], sort=False)
# Com o dataframe agrupado, a função agg vai trazer o n_volta máximo de cada piloto
# e também a soma de tempo_volta de cada piloto
classificacao = agrupado.agg({'n_volta': 'max', 'tempo_volta': 'sum'})
# Ordenando o novo dataframe por tempo_volta, que agora corresponde a soma de todas as voltas.
classificacao = classificacao.sort_values(by='tempo_volta')
classificacao

,,n_volta,tempo_volta
n_piloto,piloto,,
038,F.MASSA,4,00:04:11.578000
002,K.RAIKKONEN,4,00:04:15.153000
033,R.BARRICHELLO,4,00:04:16.080000
023,M.WEBBER,4,00:04:17.722000
015,F.ALONSO,4,00:04:54.221000
011,S.VETTEL,3,00:06:27.276000


In [63]:
# Adicionando a coluna de classificação, em ordem porque o dataframe está ordenado do melhor 
# para o pior tempo.
classificacao['posicao'] = [1, 2, 3, 4, 5 ,6]
classificacao

,,n_volta,tempo_volta,posicao
n_piloto,piloto,,,
038,F.MASSA,4,00:04:11.578000,1
002,K.RAIKKONEN,4,00:04:15.153000,2
033,R.BARRICHELLO,4,00:04:16.080000,3
023,M.WEBBER,4,00:04:17.722000,4
015,F.ALONSO,4,00:04:54.221000,5
011,S.VETTEL,3,00:06:27.276000,6


In [64]:
# Renomeando as colunas e índices do dataframe para deixar os nome mais claros.
classificacao.rename(columns={'n_volta': 'Qtde Voltas Completadas', 'tempo_volta': 'Total de Prova',
                              'posicao': 'Posição Chegada'}, inplace='True')
classificacao.index.names = ['Código Piloto', 'Nome Piloto']

In [65]:
# resultado.
classificacao

,,Qtde Voltas Completadas,Total de Prova,Posição Chegada
Código Piloto,Nome Piloto,,,
038,F.MASSA,4,00:04:11.578000,1
002,K.RAIKKONEN,4,00:04:15.153000,2
033,R.BARRICHELLO,4,00:04:16.080000,3
023,M.WEBBER,4,00:04:17.722000,4
015,F.ALONSO,4,00:04:54.221000,5
011,S.VETTEL,3,00:06:27.276000,6


## Bônus
* Descobrir a melhor volta de cada piloto

In [66]:
# Utilizando o mesmo dataframe agrupado, pegando o menor tempo de cada piloto
melhor_volta = agrupado.agg({'tempo_volta': 'min'})

In [67]:
# resultado.
melhor_volta

,,tempo_volta
n_piloto,piloto,
038,F.MASSA,00:01:02.769000
033,R.BARRICHELLO,00:01:03.716000
002,K.RAIKKONEN,00:01:03.076000
023,M.WEBBER,00:01:04.216000
015,F.ALONSO,00:01:07.011000
011,S.VETTEL,00:01:18.097000


## Bônus
* Descobrir a melhor volta da corrida

In [68]:
# Resultado 1
# Filtrando somente o registro da melhor volta
df.where(df['tempo_volta'] == df['tempo_volta'].min()).dropna()

,hora,n_piloto,piloto,n_volta,tempo_volta,velocidade_media_volta
11,23:51:14.216,038,F.MASSA,3.0,00:01:02.769000,"44,334"


In [69]:
# Resultado 2
# Neste caso como o dataframe não é muito grande, é possível ordena-lo e ver as melhores voltas ordenas
# ordenando o dataframe por tempo, do menor para o maior.
df.sort_values(by='tempo_volta')

,hora,n_piloto,piloto,n_volta,tempo_volta,velocidade_media_volta
11,23:51:14.216,038,F.MASSA,3,00:01:02.769000,"44,334"
17,23:52:17.003,038,F.MASSA,4,00:01:02.787000,"44,321"
1,23:49:08.277,038,F.MASSA,1,00:01:02.852000,"44,275"
19,23:52:22.120,002,K.RAIKKONEN,4,00:01:03.076000,"44,118"
6,23:50:11.447,038,F.MASSA,2,00:01:03.170000,"44,053"
12,23:51:18.576,033,R.BARRICHELLO,3,00:01:03.716000,"43,675"
8,23:50:15.057,002,K.RAIKKONEN,2,00:01:03.982000,"43,493"
13,23:51:19.044,002,K.RAIKKONEN,3,00:01:03.987000,"43,49"
7,23:50:14.860,033,R.BARRICHELLO,2,00:01:04.002000,"43,48"
18,23:52:22.586,033,R.BARRICHELLO,4,00:01:04.010000,"43,474"


## Bônus
* Calcular a velocidade média de cada piloto durante toda corrida

In [81]:
# Agrupando em um novo dataframe
agrupado = df.groupby(['n_piloto', 'piloto'], sort=False)
# Com o dataframe agrupado, mean traz a média da coluna velocidade_media_volta, ou seja,
# a velocidade média de cada piloto na corrida.
vel_media = agrupado.agg({'velocidade_media_volta': 'mean'})

In [82]:
# Resultado
vel_media

,,velocidade_media_volta
n_piloto,piloto,
038,F.MASSA,44.245750
033,R.BARRICHELLO,43.468000
002,K.RAIKKONEN,43.627250
023,M.WEBBER,43.191250
015,F.ALONSO,38.066250
011,S.VETTEL,25.745667


## Bônus
* Descobrir quanto tempo cada piloto chegou após o vencedor

In [83]:
# Extraindo o tempo do vencedor. (O dataframe classificacao foi ordenado por melhor tempo de prova)
melhor_tempo = classificacao['Total de Prova'].iloc[0]
# Adicionando a coluna Tempo após vencedor com a diferença do tempo de cada piloto para o primeiro.
classificacao['Tempo após vencedor'] = classificacao['Total de Prova'] - melhor_tempo

In [84]:
# Resultado
classificacao

,,Qtde Voltas Completadas,Total de Prova,Posição Chegada,Tempo após vencedor
Código Piloto,Nome Piloto,,,,
038,F.MASSA,4,00:04:11.578000,1,00:00:00
002,K.RAIKKONEN,4,00:04:15.153000,2,00:00:03.575000
033,R.BARRICHELLO,4,00:04:16.080000,3,00:00:04.502000
023,M.WEBBER,4,00:04:17.722000,4,00:00:06.144000
015,F.ALONSO,4,00:04:54.221000,5,00:00:42.643000
011,S.VETTEL,3,00:06:27.276000,6,00:02:15.698000
